In [1]:
# Install Model maker
!pip install -q tflite-model-maker

     |████████████████████████████████| 501kB 5.8MB/s 
     |████████████████████████████████| 706kB 11.8MB/s 
     |████████████████████████████████| 92kB 8.7MB/s 
     |████████████████████████████████| 122kB 16.7MB/s 
     |████████████████████████████████| 1.0MB 17.1MB/s 
     |████████████████████████████████| 1.2MB 13.1MB/s 
     |████████████████████████████████| 645kB 30.8MB/s 
     |████████████████████████████████| 112kB 51.1MB/s 
     |████████████████████████████████| 5.5MB 51.2MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 174kB 50.5MB/s 
     |████████████████████████████████| 849kB 53.4MB/s 
     |████████████████████████████████| 194kB 44.5MB/s 
     |████████████████████████████████| 37.6MB 134kB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
     |████████████████████████████████| 358kB 37.9MB/s 


In [2]:
# Imports and check that we are using TF2.x
import numpy as np
import os

from tflite_model_maker import configs
from tflite_model_maker import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker import TextClassifierDataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [3]:
# Download the data CSV
data_file = tf.keras.utils.get_file(fname='binaryemotion.csv', origin='https://storage.googleapis.com/laurencemoroney-blog.appspot.com/binary-emotion-withheaders.csv')
#For a bigger, more complex dataset, you can try toxicity
#data_file = tf.keras.utils.get_file(fname='toxicitytraining2.csv', origin='https://storage.googleapis.com/laurencemoroney-blog.appspot.com/toxicitytraining2.csv')
print(data_file)

2695168/2690517 [==============================] - 0s 0us/step
/root/.keras/datasets/binaryemotion.csv


In [4]:
# Use a model spec from model maker. Options are 'mobilebert_classifier', 'bert_classifier' and 'average_word_vec'
# The first 2 use the BERT model, which is accurate, but larger and slower to train
# Average Word Vec is kinda like transfer learning where there are pre-trained word weights
# and dictionaries
spec = model_spec.get('average_word_vec')
spec.num_words = 2000
spec.seq_len = 20
spec.wordvec_dim = 7

In [5]:
# Load the CSV using DataLoader.from_csv to make the training_data
train_data = TextClassifierDataLoader.from_csv(
      filename=os.path.join(os.path.join(data_file)),
      text_column='tweet', #For Toxicity use " value_of_text" (note the leading space)
      label_column='label', #For Toxicity also use "label"
      model_spec=spec,
      delimiter=',',
      is_training=True)

In [6]:
# Build the model
model = text_classifier.create(train_data, model_spec=spec, epochs=20)

Epoch 1/20
1103/1103 [==============================] - 6s 4ms/step - loss: 0.6497 - accuracy: 0.6401
Epoch 2/20
1103/1103 [==============================] - 4s 3ms/step - loss: 0.6653 - accuracy: 0.6039
Epoch 3/20
1103/1103 [==============================] - 4s 3ms/step - loss: 0.6067 - accuracy: 0.6647
Epoch 4/20
1103/1103 [==============================] - 3s 3ms/step - loss: 0.5906 - accuracy: 0.6851
Epoch 5/20
1103/1103 [==============================] - 4s 3ms/step - loss: 0.5844 - accuracy: 0.6922
Epoch 6/20
1103/1103 [==============================] - 4s 3ms/step - loss: 0.5811 - accuracy: 0.6953
Epoch 7/20
1103/1103 [==============================] - 4s 3ms/step - loss: 0.5770 - accuracy: 0.6976
Epoch 8/20
1103/1103 [==============================] - 4s 3ms/step - loss: 0.5758 - accuracy: 0.6987
Epoch 9/20
1103/1103 [==============================] - 4s 3ms/step - loss: 0.5734 - accuracy: 0.7006
Epoch 10/20
1103/1103 [==============================] - 3s 3ms/step - loss: 0.572

In [9]:
# Save the TFLite converted model
model.export(export_dir='/mm_emotion/')

Finished populating metadata and associated file to the model:
/mm_emotion/model.tflite
The metadata json file has been saved to:
/mm_emotion/model.json
The associated file that has been been packed to the model is:
['labels.txt', 'vocab.txt']


/usr/local/lib/python3.7/dist-packages/tensorflow_lite_support/metadata/python/metadata.py:344: UserWarning: File, 'vocab.txt', does not exsit in the metadata. But packing it to tflite model is still allowed.
  "tflite model is still allowed.".format(f))


In [8]:
# Alternatively you can shrink and quantize the model prior to exporting
config = configs.QuantizationConfig.create_dynamic_range_quantization(optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_LATENCY])
config.experimental_new_quantizer = True
model.export(export_dir='/mm_emotion/', quantization_config=config)

Finished populating metadata and associated file to the model:
/mm_emotion/model.tflite
The metadata json file has been saved to:
/mm_emotion/model.json
The associated file that has been been packed to the model is:
['labels.txt', 'vocab.txt']


/usr/local/lib/python3.7/dist-packages/tensorflow_lite_support/metadata/python/metadata.py:344: UserWarning: File, 'vocab.txt', does not exsit in the metadata. But packing it to tflite model is still allowed.
  "tflite model is still allowed.".format(f))


In [10]:
# If you want the labels and the vocab, for example for iOS, you can use this
model.export(export_dir='/mm_emotion/', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])

In [ ]:
model.summary()